In [ ]:
%pip install --upgrade --quiet  dashscope
%pip install -qU zhipuai
%pip show pydantic

In [104]:
from langchain_community.llms import Tongyi
from dotenv import load_dotenv
import os
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
# 加载 .env 文件
load_dotenv()  # 默认会加载位于项目根目录的 .env 文件

True

In [105]:
llm = Tongyi(dashscope_api_key=os.getenv("DASHSCOPE_API_KEY"), model_name="qwen-plus")
print(llm.model_name)

qwen-plus


In [106]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage
chatLLM = ChatTongyi(
    streaming=True,
)
messages = [
    ("system", "你是一名专业的翻译家，可以将用户的中文翻译为英文。"),
    ("human", "我喜欢编程。"),
]
res = chatLLM.stream([HumanMessage(content="你好请跟我进行十次对话")], streaming=True)
for r in res:
    print("chat resp:", r)

chat resp: content='当然可以'
chat resp: content='，'
chat resp: content='很高兴'
chat resp: content='和您进行对话'
chat resp: content='。请问我们从'
chat resp: content='什么话题开始呢'
chat resp: content='？'
chat resp: content=''


In [107]:
messages = [
    SystemMessage(content="Return only a JSON object as a response with no explanation text"),
    HumanMessage(content="Generate a JSON response object containing a brief description and release year for the movie '上甘岭'")
    
]

response = chatLLM(messages)
print(response)

content='{\n  "description": "A Chinese war film depicting the Battle of Triangle Hill during the Korean War.",\n  "release_year": 1956\n}'


In [108]:
messages2 = [
    SystemMessage(content="只返回一个JSON对象作为响应，不包含解释文本"),
    HumanMessage(content="生成一个JSON响应对象，其中包含电影《上甘岭》的简要描述和发行年份。")
]

response2 = chatLLM(messages2)
print(response2)

content='```json\n{\n  "title": "上甘岭",\n  "description": "《上甘岭》是一部描绘抗美援朝战争中，中国军队在上甘岭战役中的英勇表现的电影。",\n  "release_year": 1956\n}\n```'


In [109]:
batch_messages = [
    [
        SystemMessage(content="Return only a JSON object as a response with no explanation text"),
        HumanMessage(content="Generate a JSON response object containing a brief description and release year for the movie 'Inception'")
    ],
    [
        SystemMessage(content="Return only a JSON object as a response with no explanation text"),
        HumanMessage(content="Generate a JSON response object containing a brief description and release year for the movie 'Avatar'")
    ]
]

result = chatLLM.generate(batch_messages)
print(result)

generations=[[ChatGeneration(text='{\n  "movie": {\n    "title": "Inception",\n    "description": "A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO.",\n    "release_year": 2010\n  }\n}', generation_info={'finish_reason': 'stop', 'request_id': '6320b9eb-9a29-97b3-b385-21873d52e265', 'token_usage': {'input_tokens': 44, 'output_tokens': 62, 'total_tokens': 106}}, message=AIMessage(content='{\n  "movie": {\n    "title": "Inception",\n    "description": "A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO.",\n    "release_year": 2010\n  }\n}'))], [ChatGeneration(text='{\n  "movie": "Avatar",\n  "description": "A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.",\n  "release_year": 2009\n}

In [110]:
import os
from zhipuai import ZhipuAI
api_key = os.getenv("ZHIPU_API_KEY") 
print(api_key) 
client = ZhipuAI(api_key=api_key) 
response = client.embeddings.create(
    model="embedding-2", #填写需要调用的模型编码
    input="你好",
)
print(response)
embeddings = response.data[0].embedding
print(embeddings)

3730d082629c2d29999869af815e8308.RdE2Y20gmYkQwUuR
EmbeddingsResponded(object='list', data=[Embedding(object='embedding', index=0, embedding=[-0.02675455, 0.019045718, 0.006664881, -0.023293389, 0.0015206569, -0.04486882, -0.042575654, -0.0068752086, 0.041466173, 0.063009225, -0.02054925, 0.02197032, 0.010770192, 0.029588338, -0.014666274, -0.0125894295, 0.0041863034, 0.03300969, 0.00084172003, 0.03833367, 0.011762664, 0.015289724, -0.02257108, -0.0042375927, 0.004333133, -0.020114154, 0.0082468465, -0.029370125, -0.028433055, -0.03754437, 0.020863462, 0.0025333231, -0.010941108, 0.014648178, 0.023267869, 0.054538768, -0.0021926227, 0.045446496, 0.03306297, -0.029986374, -0.039381802, 0.0035400314, -0.01769706, -0.058106888, 0.02672521, 0.008502675, 0.041842893, 0.04273137, 0.027076239, 0.0016063545, 0.015132259, 0.0006472097, 0.0018888881, 0.011831868, 0.029406136, 0.035970062, 0.03429845, 0.011271943, -0.026558371, 0.0523376, -0.027810996, 0.030679826, -0.025549809, 0.023770476, 0.021

In [ ]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"
vector_store = InMemoryVectorStore(embeddings)
vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content